# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from keys.api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,27.83,46,20,0.00,CA,1693255583
1,1,severo-yeniseyskiy,60.3725,93.0408,12.89,88,100,2.05,RU,1693256023
2,2,laguna,38.4210,-121.4238,37.41,28,0,2.57,US,1693256023
3,3,tiksi,71.6872,128.8694,6.38,88,100,0.46,RU,1693256023
4,4,mugia,43.1041,-9.2179,18.84,84,93,5.30,ES,1693256023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      tiles = "OSM",
                                      frame_width = 700,
                                      frame_height = 500,
                                      size = "Humidity",
                                      scale = 0.5,
                                      color = "City",
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,antsohihy,MG,-14.8796,47.9875,60,
118,huangpi,CN,30.8845,114.3779,83,
135,petra,JO,30.3222,35.4793,53,
137,grafton,US,42.2070,-71.6856,73,
160,prince rupert,CA,54.3161,-130.3201,77,
222,constantia,RO,44.1833,28.6500,57,
273,khorramabad,IR,33.4878,48.3558,18,
279,suez,EG,29.9737,32.5263,72,
323,cururupu,BR,-1.8283,-44.8683,77,
368,luderitz,NaN,-26.6481,15.1594,24,


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# My criteria

# A max temperature lower than 28 degrees but higher than 20
# Cloudiness less than 3
# Wind speed less than 4.5 m/s

ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 28)
                                    & (city_data_df["Max Temp"] > 20) 
                                    & (city_data_df["Cloudiness"] < 3)
                                    & (city_data_df["Wind Speed"] < 4.5)
                                     ]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,antsohihy,-14.8796,47.9875,22.81,60,0,4.29,MG,1693256026
118,118,huangpi,30.8845,114.3779,23.06,83,0,2.00,CN,1693256039
135,135,petra,30.3222,35.4793,20.25,53,0,2.45,JO,1693255910
137,137,grafton,42.2070,-71.6856,24.15,73,0,3.60,US,1693255879
160,160,prince rupert,54.3161,-130.3201,20.05,77,0,3.09,CA,1693256049


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:


# Set parameters to search for a hotel
radius = 5000  
params = {
    "apiKey": "geoapify_key", 
    "types": "hotel",
    "limit": 1
}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # You can adjust the API endpoint based on what's provided in the documentation
    base_url = f"https://api.geoapify.com/v2/places?lat={lat}&lon={lng}&radius={radius}"

    response = requests.get(base_url, params=params)
    data = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df


Starting hotel search
antsohihy - nearest hotel: No hotel found
huangpi - nearest hotel: No hotel found
petra - nearest hotel: No hotel found
grafton - nearest hotel: No hotel found
prince rupert - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
khorramabad - nearest hotel: No hotel found
suez - nearest hotel: No hotel found
cururupu - nearest hotel: No hotel found
luderitz - nearest hotel: No hotel found
marapanim - nearest hotel: No hotel found
damghan - nearest hotel: No hotel found
tantou - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
fort st. john - nearest hotel: No hotel found
garrucha - nearest hotel: No hotel found
ibanda - nearest hotel: No hotel found
novooleksiyivka - nearest hotel: No hotel found
jayrud - nearest hotel: No hotel found
myrina - nearest hotel: No hotel found
gazojak - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
28,antsohihy,MG,-14.8796,47.9875,60,No hotel found
118,huangpi,CN,30.8845,114.3779,83,No hotel found
135,petra,JO,30.3222,35.4793,53,No hotel found
137,grafton,US,42.2070,-71.6856,73,No hotel found
160,prince rupert,CA,54.3161,-130.3201,77,No hotel found
222,constantia,RO,44.1833,28.6500,57,No hotel found
273,khorramabad,IR,33.4878,48.3558,18,No hotel found
279,suez,EG,29.9737,32.5263,72,No hotel found
323,cururupu,BR,-1.8283,-44.8683,77,No hotel found
368,luderitz,NaN,-26.6481,15.1594,24,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   tiles = "OSM",
                                   frame_width = 700,
                                   frame_height = 500,
                                   size = "Humidity",
                                   color = "City",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)